# Final Model Training and Evaluation

This notebook performs final model training using the best model selected from model comparison.

## Objectives:
- Load the best model configuration
- Train on the full training dataset
- Evaluate on test set
- Generate final predictions
- Create model performance report
- Save final model artifacts

## 1. Import Required Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import json
import joblib
import os
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve
)

# Model imports (will be loaded based on metadata)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Utilities
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 2. Load Data and Previous Results

In [ ]:
# Load datasets
train_data = pd.read_csv('../data/raw/train_data.csv')
train_labels = pd.read_csv('../data/raw/train_labels.csv')
test_data = pd.read_csv('../data/raw/test_data.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test data shape: {test_data.shape}")

# Get target column
target_column = train_labels.columns[-1]
print(f"Target column: {target_column}")

In [ ]:
# Load model metadata from previous selection
try:
    with open('../models/model_metadata.json', 'r') as f:
        model_metadata = json.load(f)
    
    print("Loaded model metadata:")
    print(f"- Best model: {model_metadata['model_name']}")
    print(f"- Model type: {model_metadata['model_type']}")
    print(f"- Validation accuracy: {model_metadata['validation_accuracy']:.4f}")
    print(f"- Features used: {model_metadata['features_used']}")
    
except FileNotFoundError:
    print("No model metadata found. Please run model selection notebook first.")
    model_metadata = None

In [ ]:
# Load problematic features if available
try:
    with open('../data/processed/problematic_features.json', 'r') as f:
        problematic_features = json.load(f)
    print(f"Loaded problematic features info")
except FileNotFoundError:
    print("No problematic features file found")
    problematic_features = {'constant_features': [], 'low_variance_features': []}

## 3. Data Preprocessing (Consistent with Model Selection)

In [ ]:
# Apply same preprocessing as in model selection
features_to_remove = set(
    problematic_features.get('constant_features', []) + 
    problematic_features.get('low_variance_features', [])
)

# Prepare training data
good_features = [col for col in train_data.columns if col not in features_to_remove]
X_full = train_data[good_features].copy()
y_full = train_labels[target_column].copy()

# Prepare test data
X_test = test_data[good_features].copy()

print(f"Features after cleaning: {X_full.shape[1]}")
print(f"Training samples: {X_full.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Handle missing values
if X_full.isnull().sum().sum() > 0:
    print("Filling missing values...")
    X_full = X_full.fillna(X_full.median())
    X_test = X_test.fillna(X_full.median())  # Use training median for test

# Encode target if needed
le = LabelEncoder()
if y_full.dtype == 'object':
    y_encoded = le.fit_transform(y_full)
    print(f"Target encoded: {dict(zip(le.classes_, range(len(le.classes_))))}")
else:
    y_encoded = y_full.values
    le = None

## 4. Feature Scaling and Selection

In [ ]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)
X_test_scaled = scaler.transform(X_test)

print(f"Feature scaling completed")

# Feature selection (use same number as in model selection)
if model_metadata:
    k_features = model_metadata['features_used']
else:
    k_features = min(1000, X_scaled.shape[1])

selector = SelectKBest(score_func=f_classif, k=k_features)
X_selected = selector.fit_transform(X_scaled, y_encoded)
X_test_selected = selector.transform(X_test_scaled)

selected_features = X_full.columns[selector.get_support()]
print(f"Selected {k_features} features for training")
print(f"Final training shape: {X_selected.shape}")
print(f"Final test shape: {X_test_selected.shape}")

## 5. Load or Create Final Model

In [ ]:
# Try to load the best model from model selection
if model_metadata:
    model_name = model_metadata['model_name']
    model_file = f"../models/best_model_{model_name.lower().replace(' ', '_')}.pkl"
    
    try:
        best_model = joblib.load(model_file)
        print(f"Loaded saved model: {model_name}")
        print(f"Model parameters: {best_model.get_params()}")
        
    except FileNotFoundError:
        print(f"Saved model not found. Creating new {model_name} with best parameters.")
        
        # Create model with best hyperparameters from metadata
        if model_metadata['model_type'] == 'RandomForestClassifier':
            best_model = RandomForestClassifier(**model_metadata['hyperparameters'])
        elif model_metadata['model_type'] == 'XGBClassifier':
            best_model = xgb.XGBClassifier(**model_metadata['hyperparameters'])
        elif model_metadata['model_type'] == 'LGBMClassifier':
            best_model = lgb.LGBMClassifier(**model_metadata['hyperparameters'])
        elif model_metadata['model_type'] == 'LogisticRegression':
            best_model = LogisticRegression(**model_metadata['hyperparameters'])
        else:
            print(f"Unknown model type: {model_metadata['model_type']}")
            best_model = RandomForestClassifier(random_state=42)
            
else:
    print("No model metadata available. Using default RandomForest.")
    best_model = RandomForestClassifier(n_estimators=200, random_state=42)
    model_name = "Random Forest (Default)"

## 6. Final Model Training

In [ ]:
print(f"Training final model: {type(best_model).__name__}")
print(f"Training on {X_selected.shape[0]} samples with {X_selected.shape[1]} features")

# Train the model
start_time = datetime.now()
best_model.fit(X_selected, y_encoded)
training_time = datetime.now() - start_time

print(f"Training completed in: {training_time}")
print(f"Model trained successfully!")

## 7. Learning Curves Analysis

In [ ]:
# Generate learning curves
print("Generating learning curves...")

train_sizes = np.linspace(0.1, 1.0, 10)
train_sizes_abs, train_scores, val_scores = learning_curve(
    best_model, X_selected, y_encoded, 
    train_sizes=train_sizes, cv=5, scoring='accuracy',
    random_state=42, n_jobs=-1
)

# Calculate means and stds
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 8))
plt.plot(train_sizes_abs, train_mean, 'o-', color='blue', label='Training Accuracy')
plt.fill_between(train_sizes_abs, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')

plt.plot(train_sizes_abs, val_mean, 'o-', color='red', label='Validation Accuracy')
plt.fill_between(train_sizes_abs, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title(f'Learning Curves - {type(best_model).__name__}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final training accuracy: {train_mean[-1]:.4f} (+/- {train_std[-1]:.4f})")
print(f"Final validation accuracy: {val_mean[-1]:.4f} (+/- {val_std[-1]:.4f})")

## 8. Feature Importance Analysis

In [ ]:
# Feature importance analysis
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Save feature importance
    feature_importance.to_csv('../data/processed/final_feature_importance.csv', index=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 10))
    top_20_features = feature_importance.head(20)
    plt.barh(range(len(top_20_features)), top_20_features['importance'])
    plt.yticks(range(len(top_20_features)), top_20_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 20 Feature Importances - {type(best_model).__name__}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("Top 10 most important features:")
    display(feature_importance.head(10))
    
    # Feature importance statistics
    print(f"\nFeature importance statistics:")
    print(f"Mean importance: {feature_importance['importance'].mean():.6f}")
    print(f"Std importance: {feature_importance['importance'].std():.6f}")
    print(f"Max importance: {feature_importance['importance'].max():.6f}")
    print(f"Min importance: {feature_importance['importance'].min():.6f}")
    
else:
    print(f"Feature importance not available for {type(best_model).__name__}")
    feature_importance = None

## 9. Generate Test Predictions

In [ ]:
# Generate predictions on test set
print("Generating predictions on test set...")

# Predictions
test_predictions = best_model.predict(X_test_selected)
test_probabilities = best_model.predict_proba(X_test_selected)

print(f"Generated predictions for {len(test_predictions)} test samples")

# Convert back to original labels if necessary
if le is not None:
    test_predictions_original = le.inverse_transform(test_predictions)
    print(f"Converted predictions back to original labels")
else:
    test_predictions_original = test_predictions

# Create predictions DataFrame
predictions_df = pd.DataFrame({
    'sample_id': range(len(test_predictions)),
    'predicted_class': test_predictions_original
})

# Add probability columns
for i, class_name in enumerate(np.unique(y_encoded)):
    if le is not None:
        original_class = le.inverse_transform([class_name])[0]
        predictions_df[f'prob_class_{original_class}'] = test_probabilities[:, i]
    else:
        predictions_df[f'prob_class_{class_name}'] = test_probabilities[:, i]

print("Prediction distribution:")
print(predictions_df['predicted_class'].value_counts())

display(predictions_df.head(10))

## 10. Model Performance Analysis

In [ ]:
# Training set performance (for reference)
train_predictions = best_model.predict(X_selected)
train_accuracy = accuracy_score(y_encoded, train_predictions)
train_f1 = f1_score(y_encoded, train_predictions, average='weighted')

print("=== MODEL PERFORMANCE SUMMARY ===")
print(f"Model: {type(best_model).__name__}")
print(f"Training Samples: {X_selected.shape[0]}")
print(f"Features Used: {X_selected.shape[1]}")
print(f"Training Time: {training_time}")
print(f"\nTraining Performance:")
print(f"  Accuracy: {train_accuracy:.4f}")
print(f"  F1-Score: {train_f1:.4f}")

if model_metadata:
    print(f"\nValidation Performance (from model selection):")
    print(f"  Accuracy: {model_metadata['validation_accuracy']:.4f}")
    print(f"  F1-Score: {model_metadata['validation_f1']:.4f}")

print(f"\nTest Predictions Generated: {len(test_predictions)}")
print(f"Test Prediction Distribution: {dict(zip(*np.unique(test_predictions_original, return_counts=True)))}")

## 11. Prediction Confidence Analysis

In [ ]:
# Analyze prediction confidence
max_probabilities = np.max(test_probabilities, axis=1)
confidence_stats = pd.Series(max_probabilities).describe()

print("Prediction Confidence Statistics:")
print(confidence_stats)

# Plot confidence distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(max_probabilities, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Maximum Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Confidence')
plt.axvline(confidence_stats['mean'], color='red', linestyle='--', label=f'Mean: {confidence_stats["mean"]:.3f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(max_probabilities)
plt.ylabel('Maximum Probability')
plt.title('Prediction Confidence Box Plot')

plt.tight_layout()
plt.show()

# High and low confidence predictions
high_confidence_threshold = 0.9
low_confidence_threshold = 0.6

high_confidence_count = np.sum(max_probabilities >= high_confidence_threshold)
low_confidence_count = np.sum(max_probabilities <= low_confidence_threshold)

print(f"\nConfidence Analysis:")
print(f"High confidence predictions (≥{high_confidence_threshold}): {high_confidence_count} ({high_confidence_count/len(max_probabilities)*100:.1f}%)")
print(f"Low confidence predictions (≤{low_confidence_threshold}): {low_confidence_count} ({low_confidence_count/len(max_probabilities)*100:.1f}%)")
print(f"Medium confidence predictions: {len(max_probabilities) - high_confidence_count - low_confidence_count} ({(len(max_probabilities) - high_confidence_count - low_confidence_count)/len(max_probabilities)*100:.1f}%)")

## 12. Save Final Results

In [ ]:
# Save predictions
predictions_df.to_csv('../data/processed/test_predictions.csv', index=False)
print("Test predictions saved to ../data/processed/test_predictions.csv")

# Save final model and preprocessing objects
model_filename = f"final_model_{type(best_model).__name__.lower()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(best_model, f'../models/{model_filename}')
joblib.dump(scaler, '../models/final_scaler.pkl')
joblib.dump(selector, '../models/final_feature_selector.pkl')

if le is not None:
    joblib.dump(le, '../models/final_label_encoder.pkl')

print(f"Final model saved as: {model_filename}")

# Create comprehensive model report
final_report = {
    'model_info': {
        'model_name': type(best_model).__name__,
        'model_parameters': best_model.get_params(),
        'training_date': datetime.now().isoformat(),
        'training_time_seconds': training_time.total_seconds()
    },
    'data_info': {
        'training_samples': X_selected.shape[0],
        'features_used': X_selected.shape[1],
        'total_original_features': X_full.shape[1],
        'test_samples': X_test_selected.shape[0],
        'target_classes': int(len(np.unique(y_encoded))),
        'feature_selection_method': 'SelectKBest with f_classif'
    },
    'performance': {
        'training_accuracy': float(train_accuracy),
        'training_f1_weighted': float(train_f1),
        'learning_curve_final_train': float(train_mean[-1]),
        'learning_curve_final_val': float(val_mean[-1])
    },
    'predictions': {
        'test_samples_predicted': int(len(test_predictions)),
        'prediction_distribution': {str(k): int(v) for k, v in zip(*np.unique(test_predictions_original, return_counts=True))},
        'confidence_stats': {
            'mean_confidence': float(confidence_stats['mean']),
            'std_confidence': float(confidence_stats['std']),
            'min_confidence': float(confidence_stats['min']),
            'max_confidence': float(confidence_stats['max']),
            'high_confidence_count': int(high_confidence_count),
            'low_confidence_count': int(low_confidence_count)
        }
    },
    'files_created': {
        'model': model_filename,
        'scaler': 'final_scaler.pkl',
        'feature_selector': 'final_feature_selector.pkl',
        'label_encoder': 'final_label_encoder.pkl' if le is not None else None,
        'predictions': 'test_predictions.csv',
        'feature_importance': 'final_feature_importance.csv' if feature_importance is not None else None
    }
}

# Add feature importance to report if available
if feature_importance is not None:
    final_report['feature_importance'] = {
        'top_10_features': feature_importance.head(10).to_dict('records'),
        'importance_stats': {
            'mean': float(feature_importance['importance'].mean()),
            'std': float(feature_importance['importance'].std()),
            'max': float(feature_importance['importance'].max()),
            'min': float(feature_importance['importance'].min())
        }
    }

# Save final report
with open('../models/final_model_report.json', 'w') as f:
    json.dump(final_report, f, indent=2)

print("Final model report saved to ../models/final_model_report.json")

## 13. Final Summary

In [ ]:
print("" + "="*60)
print("FINAL MODEL TRAINING COMPLETED SUCCESSFULLY")
print("="*60)

print(f"\n🤖 Model Details:")
print(f"   Type: {type(best_model).__name__}")
print(f"   Training Time: {training_time}")
print(f"   Features Used: {X_selected.shape[1]:,} out of {X_full.shape[1]:,}")

print(f"\n📊 Performance:")
print(f"   Training Accuracy: {train_accuracy:.4f}")
print(f"   Training F1-Score: {train_f1:.4f}")
if model_metadata:
    print(f"   Validation Accuracy: {model_metadata['validation_accuracy']:.4f}")

print(f"\n🎯 Predictions:")
print(f"   Test Samples: {len(test_predictions):,}")
print(f"   Mean Confidence: {confidence_stats['mean']:.3f}")
print(f"   High Confidence (≥90%): {high_confidence_count:,} ({high_confidence_count/len(max_probabilities)*100:.1f}%)")

print(f"\n💾 Files Created:")
print(f"   📁 ../models/{model_filename}")
print(f"   📁 ../models/final_scaler.pkl")
print(f"   📁 ../models/final_feature_selector.pkl")
print(f"   📁 ../models/final_model_report.json")
print(f"   📁 ../data/processed/test_predictions.csv")
if feature_importance is not None:
    print(f"   📁 ../data/processed/final_feature_importance.csv")

print(f"\n✅ Next Steps:")
print(f"   1. Review prediction results and confidence scores")
print(f"   2. Validate model performance on additional data if available")
print(f"   3. Deploy model for production use")
print(f"   4. Monitor model performance over time")
print(f"   5. Consider retraining with new data")

print("\n" + "="*60)
print("🎉 PROJECT COMPLETED SUCCESSFULLY!")
print("="*60)